In [1]:
import torch
import torch.nn as nn
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm 

In [2]:
# koELECTRA 토크나이저 불러오기
tokenizer = ElectraTokenizer.from_pretrained("koelectra-base-v3-discriminator")

In [3]:
''' 모델가중치를 불러오거나 체크포인트를 불러올 때 실행 '''

# 모델의 상태 딕셔너리를 로드합니다.
model_state_dict = torch.load("Ternary_model_state_dict_epoch_15.pt")

# 모델을 생성하고 상태를 로드합니다.
model = ElectraForSequenceClassification.from_pretrained("koelectra-base-v3-discriminator", num_labels=3)
model.load_state_dict(model_state_dict)

# 옵티마이저의 상태 딕셔너리를 로드합니다.
optimizer_state_dict = torch.load("Ternary_optimizer_state_dict_epoch_15.pt")

Some weights of the model checkpoint at koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at koelectra-base-v3-discriminator and are newly initialized: ['classifier.out_p

In [4]:
# 변경하고자 하는 Dropout 비율
new_dropout_rate = 0.2

# 모든 Dropout 레이어의 비율 변경
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = new_dropout_rate


In [ ]:
# 모델 구조 확인
print(model)

In [ ]:
# 장치 설정 (GPU 사용을 위해)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 실제로 예측해보기

In [6]:
# 한국어 문장을 입력으로 받아서 예측 라벨을 출력하는 함수
def predict_label(sentence, model, tokenizer, device):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()
        return predicted_label

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("../dataset/2018산림복지_본문_맞춤법검사.txt", encoding='UTF-8')
df

,Unnamed: 0,split_str,org_idx,correct_str
0,0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다,0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.
1,1,momo 2018 12 31 19 52 https blog naver com omj...,0,"mom 2018, 12, 31, 19, 52 https blog NAVER com ..."
2,2,An enemy generally says and believes what he w...,0,An enemy generally says and believes what he w...
3,3,True love is the joy of life 진실한 사랑은 인생의 환희다,0,True love is the joy of life 진실한 사랑은 인생의 환희다
4,4,Carpe diem 현재를 즐겨라,0,Crape idem 현재를 즐겨라.
...,...,...,...,...
135819,135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다,5170,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.
135820,135820,조은영 원광대 교수 미술과 는 2017년을 종합하기에는 수락석출 외의 단어들이 지나...,5170,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...
135821,135821,올해의 사자성어는 3위부터 5위까지는 약 16 대의 고른 분포를 보인 것이 특징이다,5170,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.
135822,135822,4위는 16 5위는 15 1 였다,5170,"4위는 16, 5위는 15, 1 쳤다"


In [9]:
# 한국어 문장 입력 받기
korean_sentences = df['correct_str'].tolist()

# 예측 라벨 출력
emotion_labels = {0:'부정', 1:'중립', 2:'긍정'}
predicted_label = [emotion_labels[predict_label(korean_sentence, model, tokenizer, device)] for korean_sentence in tqdm(korean_sentences)]
df['label'] = predicted_label

100%|██████████| 135824/135824 [1:33:04<00:00, 24.32it/s] 


In [10]:
ddff = df[['correct_str','label','org_idx']]
ddff

,correct_str,label,org_idx
0,블로그 mom 5개의 글 mom 목록 열기 영어 명언 모음입니다.,긍정,0
1,"mom 2018, 12, 31, 19, 52 https blog NAVER com ...",긍정,0
2,An enemy generally says and believes what he w...,부정,0
3,True love is the joy of life 진실한 사랑은 인생의 환희다,긍정,0
4,Crape idem 현재를 즐겨라.,긍정,0
...,...,...,...
135819,한편 경계의 의미로 이 사자성어를 추천한 이들도 눈에 띈다.,부정,5170
135820,조은영 원광대 교수 미술과는 2017년을 종합하기에는 수락 석출 외의 단어들이 지나...,부정,5170
135821,올해의 사자성어는 3위부터 5위까지는 약 16대의 고른 분포를 보인 것이 특징이다.,긍정,5170
135822,"4위는 16, 5위는 15, 1 쳤다",부정,5170


In [11]:
ddff['label'].value_counts()

긍정    94869
부정    32320
중립     8635
Name: label, dtype: int64

In [12]:
ddff[ddff['label'] == '중립']

,correct_str,label,org_idx
6,어떠세요.,중립,0
24,WHO 비교할 LED 달라 만들기 기기 전문 이바지할 계절 최근 3 phosphor...,중립,0
37,제안하세요.,중립,0
62,속에 있는 얘기 남겨주세요.,중립,1
64,속에 있는 얘기 남겨주세요.,중립,1
...,...,...,...
135712,새해 복 많이 받으세요.,중립,5164
135724,편의점 한의원 약국 학원 부동산 운동센터 치과 패스트푸드점 병원 등등 고민하다 늦지...,중립,5165
135793,아무리 추이를 따르더라도 하실 일은 하셔야지요.,중립,5170
135795,소는 밭을소는 밭을 갈고 말은 사람이 타는 법이지요.,중립,5170


In [13]:
ddff.rename(columns={'correct_str': 'sentence'}, inplace=True)
ddff.rename(columns={'label': 'predicted_sentiment'}, inplace=True)
ddff.to_csv("./Ternary_classification_2018.csv", index=False, encoding='UTF-8')

c:\Users\user\anaconda3\envs\NIFOS\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
